In [45]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup


# pd.set_option('display.max_colwidth')

In [47]:
# bad songs dataframe
bad_songs = pd.read_csv('datasets/bad_cleaned2.csv')
bad_songs.head(5)

,artist,song,lyrics
0,YG,BAND DRUM (feat. A$AP Rocky),I mix the 4Hunnid with the designer Stay lace...
1,Chris Cooq,Lactose,I came to rule this stage with that Budden swa...
2,Chris Cooq,Same - Original mix,Gorillaz - Song Machine Episode 7 - 4/4 mxmto...
3,Curbo,Debauchery - Original mix,"Learning is, indeed, a very great and a very..."
4,G Herbo,Bon appétit,"Young nigga eatin', uh, bon appetit, uh Messe..."


In [48]:
# good songs dataframe
good_songs = pd.read_csv('datasets/good_cleaned.csv')
good_songs = good_songs[['artist','song','lyrics']].copy()
good_songs.head(5)

,artist,song,lyrics
0,sam the sham and the pharaohs,wooly bully,sam the sham miscellaneous wooly bully wooly b...
1,four tops,i cant help myself sugar pie honey bunch,sugar pie honey bunch you know that i love yo...
2,we five,you were on my mind,when i woke up this morning you were on my mi...
3,the righteous brothers,youve lost that lovin feelin,you never close your eyes anymore when i kiss...
4,petula clark,downtown,when youre alone and life is making you lonel...


In [84]:
# combined dataframe
main = pd.concat([good_songs, bad_songs], ignore_index=True)

In [67]:
# text cleaning 
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove punctuation
    cleaned_text = re.sub(r'[.,!?\\-]', '', text)

    return cleaned_text

main['lyrics'] = main['lyrics'].apply(clean_text)

In [83]:
# result = main['lyrics'].str.contains('\.').sum()
main

,artist,song,lyrics
0,sam the sham and the pharaohs,wooly bully,sam the sham miscellaneous wooly bully wooly b...
1,four tops,i cant help myself sugar pie honey bunch,sugar pie honey bunch you know that i love yo...
2,we five,you were on my mind,when i woke up this morning you were on my mi...
3,the righteous brothers,youve lost that lovin feelin,you never close your eyes anymore when i kiss...
4,petula clark,downtown,when youre alone and life is making you lonel...
...,...,...,...
8334,King John,Mismatched,Weve been dispatched by God sent to warn these...
8335,Buck Meek,Cannonball!,Beats me down by the pool sweet Suzy summers g...
8336,Big Beats,Young Forever (Instrumental),Proving himself in the booth was just the be...
8337,Big Beats,Escape (Instrumental),Im just me I cant be different Though I...


In [85]:
'''splitting'''
# Create a new column 'label' with 1 for good songs and 0 for bad songs
main['label'] = (main['lyrics'].isin(good_songs['lyrics'])) * 1

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(main['lyrics'], main['label'], random_state=42)

y_test = y_test.values.reshape(-1, 1)

In [86]:
# Define the n-gram lengths to evaluate
ngram_lengths = [1, 2, 3]  # Example: Unigrams, Bigrams, Trigrams

# Iterate over different n-gram lengths
for n in ngram_lengths:
    # Create the CountVectorizer with the specified n-gram length
    vect = CountVectorizer(min_df=5, ngram_range=(1, n))

    # Apply tokenization and vectorization on the 'lyrics' column
    text_train = vect.fit_transform(X_train)
    text_test = vect.transform(X_test)

    # Create a logistic regression model
    lr = LogisticRegression()

    # Perform cross-validation on the training data and compute the mean accuracy
    scores = cross_val_score(lr, text_train, y_train, cv=5)
    mean_accuracy = scores.mean()

    # Print the results for the current n-gram length
    print(f"Accuracy with n-gram length {n}: {mean_accuracy}")

    # Fit the logistic regression model on the full training data and evaluate on the test data
    lr.fit(text_train, y_train)
    test_accuracy = lr.score(text_test, y_test)
    print(f"Test Accuracy with n-gram length {n}: {test_accuracy}")

# n-gram = 1: accuracy 0.9851294644284574, test accuracy 0.9908872901678657
# n-gram = 2: accuracy 0.9884874180655476, test accuracy 0.9932853717026379
# n-gram = 3: accuracy 0.9881676738609114, test accuracy 

c:\Users\kubad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\kubad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

Accuracy with n-gram length 1: 0.9851294644284574


c:\Users\kubad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Test Accuracy with n-gram length 1: 0.9908872901678657


c:\Users\kubad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\kubad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

...

c:\Users\kubad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Test Accuracy with n-gram length 2: 0.9932853717026379


c:\Users\kubad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\kubad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

Accuracy with n-gram length 3: 0.9881676738609114


KeyboardInterrupt: 

In [10]:
'''applying bag of words'''

# Create a CountVectorizer object
vect = CountVectorizer() #(min_df=5)

text_train = vect.fit_transform(X_train)
text_test = vect.transform(X_test)

In [11]:
print("text_train:\n{}".format(repr(text_train)))
#shape of our text_train is 8106x327664 - vocabulary contains 327664 entries (features) stored as SciPy sparse matrix
#shape after constraint to have appearance in at least 5 documents, shrinks to 108184 features

text_train:
<6675x278978 sparse matrix of type '<class 'numpy.int64'>'
	with 3696204 stored elements in Compressed Sparse Row format>


In [12]:
# Create a logistic regression object and fit the model on the training data
lr = LogisticRegression()
lr.fit(text_train, y_train)

# Evaluate the model on the test data
score = lr.score(text_test, y_test)
print("Accuracy:", score)

Accuracy: 0.9886159376872379


c:\Users\kubad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
